## Сессия 3

#### Импортирование библиотек

In [3]:
import pandas as pd
import io
import glob
import re
import nltk
import matplotlib.pyplot as plt
import pymorphy2
import fitz
import string
import numpy as np
import re
import pyLDAvis
import pyLDAvis.gensim
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm, trange
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn
from __future__ import division
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
import seaborn as sns

### Формирование структуры набора данных

__TitleCompany__ - Название компании

__TextPostLemat__ - обработанный, лематизированный текст 

__TextPostToken__ - обработанный, токенизированный и лематизированный текст (лематизирован другим способом нежели TextPostLemat)

__bigram__ - биграммы обработанного и лематизированного текста

__trigram__ - триграммы обработанного и лематизированного текста

__keyword__ - ключевые слова обработанного и лематизированного текста

__clustersAgglomerative__ - кластеры, образованные моделью Agglomerative

__clustersKmeans__ - кластеры, образованные моделью Kmeans

__clustersSpectral__ - кластеры, образованные моделью Spectral

## 3.1 Разбиение выборки

In [4]:
dfHabr1 = pd.read_csv(r'C:\Users\essww\OneDrive\Рабочий стол\WORLDSKILLS\Report2.csv')

In [5]:
dfHabr1.head()

,TitleCompany,TextPost,TextPostLemat,TextPostToken,bigram,trigram,keyword,clustersAgglomerative,clustersKmeans,clustersSpectral
0,Нетология,Многие из нас ежедневно пользуются поисковыми...,многие ежедневно пользоваться поисковый систем...,"['многие', 'ежедневно', 'пользоваться', 'поиск...","[('многие', 'ежедневно'), ('ежедневно', 'польз...","[('многие', 'ежедневно', 'пользоваться'), ('еж...","человек, язык, компьютерный, текст, лингвистик...",5,3,0
1,Gett,"Всем привет, меня зовут Антон Тупиков, я – ти...",привет звать антон тупик тимлид команда просто...,"['привет', 'звать', 'антон', 'тупик', 'тимлид'...","[('привет', 'звать'), ('звать', 'антон'), ('ан...","[('привет', 'звать', 'антон'), ('звать', 'анто...","сервис, год, поездка, решение, клиент, каждый,...",1,0,0
2,Gett,"Привет! Меня зовут Виталий Костоусов, я работ...",привет звать виталий костоусов работать команд...,"['привет', 'звать', 'виталий', 'костоусов', 'р...","[('привет', 'звать'), ('звать', 'виталий'), ('...","[('привет', 'звать', 'виталий'), ('звать', 'ви...","проблема, сервис, работать, работа, команда, м...",2,4,0
3,Gett,"Всем привет, меня зовут Лера Лисичкина, я рук...",привет звать лера лисичкин руководить технолог...,"['привет', 'звать', 'лера', 'лисичкин', 'руков...","[('привет', 'звать'), ('звать', 'лера'), ('лер...","[('привет', 'звать', 'лера'), ('звать', 'лера'...","команда, рассказывать, разработка, начинать, р...",1,0,0
4,Gett,"Всем привет, меня зовут Антон Тупиков, я – ти...",привет звать антон тупик тимлид команда просто...,"['привет', 'звать', 'антон', 'тупик', 'тимлид'...","[('привет', 'звать'), ('звать', 'антон'), ('ан...","[('привет', 'звать', 'антон'), ('звать', 'анто...","сервис, год, поездка, решение, клиент, каждый,...",1,0,0


In [6]:
X = dfHabr1['TextPost']
y = dfHabr1['clustersAgglomerative']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [27]:
my_tags = dfHabr1['clustersAgglomerative'].unique()
my_tags

array([5, 1, 2, 7, 8, 6, 0, 3, 9, 4], dtype=int64)

MultinomialNB - это алгоритм классификации, который используется в машинном обучении для классификации текста и других данных с дискретными признаками. Он является вариантом алгоритма Naive Bayes, который основан на теореме Байеса и предположении о независимости между признаками. MultinomialNB особенно полезен для задач классификации текста, таких как фильтрация спама и анализ тональности текста. Алгоритм работает путем расчета вероятности каждого класса на основе входных признаков и выбора класса с наибольшей вероятностью в качестве выхода. В случае классификации текста входными признаками обычно являются частоты каждого слова в документе, а классы - это категории, на которые можно классифицировать документ. MultinomialNB реализован в библиотеке scikit-learn на Python, которая обеспечивает простой и эффективный интерфейс для обучения и использования алгоритма. MultinomialNB выбирается, когда распределение признаков дискретное

In [9]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [10]:
%%time
nb.fit(X_train, y_train)

CPU times: total: 62.5 ms
Wall time: 90.3 ms


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [11]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

CPU times: total: 0 ns
Wall time: 4.01 ms


In [12]:
y_pred[0]

1

In [14]:
my_tags

array([3, 0, 4, 8, 7, 1, 9, 5, 6, 2], dtype=int64)

In [15]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12



In [16]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [17]:
%%time
sgd.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 39.1 ms


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [18]:
%%time
y_pred = sgd.predict(X_test)

CPU times: total: 0 ns
Wall time: 4.52 ms


In [19]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12



SGDClassifier - это алгоритм классификации, который использует стохастический градиентный спуск для обучения модели. Он является одним из наиболее эффективных алгоритмов для решения крупномасштабных и разреженных задач машинного обучения, часто встречающихся при классификации текста и обработке естественного языка. Алгоритм работает путем минимизации функции потерь с помощью градиентного спуска, который обновляет веса модели на каждом шаге, используя только один случайный пример из обучающего набора за раз. Это позволяет SGDClassifier обучаться на больших наборах данных быстрее, чем другие алгоритмы, такие как логистическая регрессия и SVM. В библиотеке scikit-learn на Python реализован SGDClassifier, который обеспечивает простой и эффективный интерфейс для обучения и использования алгоритма

Логистическая регрессия - это алгоритм машинного обучения, который используется для решения задач бинарной классификации, то есть разделения данных на два класса. Он основан на линейной регрессии и использует логистическую функцию для преобразования выхода в вероятность принадлежности к классу. Логистическая регрессия хорошо подходит для задач, в которых требуется определить, принадлежит ли объект к определенному классу или нет, например, для определения склонности к депрессии. В библиотеке scikit-learn на Python реализована логистическая регрессия, которая обеспечивает простой и эффективный интерфейс для обучения и использования алгоритма. Логистическая регрессия также может использоваться для многоклассовой классификации, когда требуется разделить данные на более чем два класса

In [20]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [21]:
%%time
logreg.fit(X_train, y_train)

CPU times: total: 531 ms
Wall time: 267 ms


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [22]:
%%time
y_pred = logreg.predict(X_test)

CPU times: total: 0 ns
Wall time: 4.51 ms


In [23]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12



In [25]:
econ_text = 'проблема, сервис, работать, работа, команда, м...'

In [26]:
ect_pred = nb.predict([econ_text])
ect_pred

array([1], dtype=int64)

## 3.2 Оптимизация модели

## 3.3 Подготовка отчета




__2.1 Поиск ключевых слов/n-грамм. Векторизация текста__ Пункт был выполнен без нареканий. Биграммы, триграммы и векторизация выполнены успешно.

__2.2 Тематическое моделирование__ Сначала была выполнена кластеризация, потом тематическое моделирование. Само тематическое моделирование было выполнено тремя способами LDA, NMF и LSA, оно прошло успешно и была выбрана LDA.

    __2.2.1 LDA__ (лучшая модель) выбрал потому, что он может быть полезен в случаях, когда нужно опредлить тематическую структуру в большом наборе документов, таких как статьи и т.д. 

    __2.2.2 NMF__ выбрал потому, что он может быть полезен в случаях, когда нужно найти скрытые признаки в данных, таких как тексты и т.д.

    __2.2.3 LSA__ выбрал потому, что он может быть полезен в случаях, когда нужно определить структуру в большом наборе текстовых данных, таких как статьи, отчеты и т.д.


__2.3 Кластеризация__ Предварительная обработка данных выполнена без проблем. Кластеры показали себя одинакого, по сравнению друг с другом из-за того, что текста довольно разные и их мало, оценка кластеризации не сработало из-за маленького количества строк, в соответствии кластеров.

    __2.3.1 Agglomerative__ выбрал потому, что он полезен в случаях, когда нужно разбить данные на группы на основе их сходства.

    __2.3.2 Kmeans__ выбрал потому, что он полезен в случаях, когда нужно разбить данные на группы на основе их сходства.

    __2.3.3 Spectral__ выбрал потому, что он полезен в случаях, когда нужно разбить данные на группы на основе их сходства, но данные не могут быть разделены на группы в исходном пространстве признаков.

__2.4 Разведочный анализ__ Предварительная обработка данных выполнена полностью, без проблем.

### Вывод

В итоге датафрейм почти полностью готов к 3-ей Сессии. Но есть небольшие недочеты, по типу того, что датасет очень маленький. И проблемы связанные с этим. Такие как проблема с кластеризацией.

